In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model="llama-3.3-70b-versatile"
)
response = llm.invoke("first ")
print(response.content)

You're first. What's on your mind? Want to chat about something in particular or just see where the conversation takes us?


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.walmart.com/us/jobs/WD1996569-software-engineer-iii")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Engineer III | Walmart Careers             















  

    Careers    All Career Areas  Back  Stores & Clubs  Sam’s Club Jobs  Sam’s Club Management Jobs  Walmart Management Jobs  Walmart Store Jobs   Distribution, Fulfillment, & Drivers  Distribution and Fulfillment Centers  Drivers & Transportation  Fleet Maintenance  Flex Associate  Manufacturing   Healthcare  Corporate Healthcare Roles  Optical  Pharmacy   Technology  Cybersecurity  Data Science and Analytics  Information Technology  Project and Program Management - Technology  Software Development and Engineering  UX Design  View all Technology   Corporate  Accounting and Finance  Administrative and Support Services  Asset Protection  Aviation and Travel  Business Operations  Corporate Affairs and Communications  Customer Service and Call Center  Data Analytics and Business Intelligence  Engineering  Ethics and Compliance  Global Investigations and Security  Human Resources  Installation, Maintenance and Utilities

In [5]:
from langchain_core.prompts import PromptTemplate

# Define the prompt
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: role, experience, skills, and description.
    Only return the valid JSON object without any additional text, preamble, markdown, or labels.
    DO NOT wrap the JSON in triple backticks or any other formatting.
    DO NOT include 'json' or any extra labels.
    Just return the pure JSON object.
    """
)

# Create the chain
chain_extract = prompt_extract | llm 

# Invoke the chain
res = chain_extract.invoke(input={'page_data': page_data})

print(res.content)

{
  "role": "Software Engineer III",
  "experience": "2 years",
  "skills": [
    "Java programming languages",
    "NoSQL databases (CosmosDB or MongoDB)",
    "Container orchestration frameworks (Kubernetes)",
    "Continuous integration and continuous deployment strategies",
    "Telemetry and observability frameworks (Prometheus, Grafana, ELK)",
    "Unit and performance testing of applications (Jenkins, Junit, JMeter)",
    "Message queue Kafka"
  ],
  "description": "Collaborate with our product team, architects, quality assurance to design, develop and enhance a reliable and easy-to-maintain infrastructure. Partner and collaborate on product development with business stakeholders and cross functional teams. Troubleshoot business and production issues by gathering information. Engage support teams to assist in the resolution of issues. Perform root cause analysis to prevent future occurrence of issues."
}


In [6]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III',
 'experience': '2 years',
 'skills': ['Java programming languages',
  'NoSQL databases (CosmosDB or MongoDB)',
  'Container orchestration frameworks (Kubernetes)',
  'Continuous integration and continuous deployment strategies',
  'Telemetry and observability frameworks (Prometheus, Grafana, ELK)',
  'Unit and performance testing of applications (Jenkins, Junit, JMeter)',
  'Message queue Kafka'],
 'description': 'Collaborate with our product team, architects, quality assurance to design, develop and enhance a reliable and easy-to-maintain infrastructure. Partner and collaborate on product development with business stakeholders and cross functional teams. Troubleshoot business and production issues by gathering information. Engage support teams to assist in the resolution of issues. Perform root cause analysis to prevent future occurrence of issues.'}

In [26]:
!pip install pandas

In [7]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
links = collection.query(query_texts=['skills'], n_results=3).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [16]:
job = json_res
job["skills"]

['Java programming languages',
 'NoSQL databases (CosmosDB or MongoDB)',
 'Container orchestration frameworks (Kubernetes)',
 'Continuous integration and continuous deployment strategies',
 'Telemetry and observability frameworks (Prometheus, Grafana, ELK)',
 'Unit and performance testing of applications (Jenkins, Junit, JMeter)',
 'Message queue Kafka']

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Abhiram Reddy Mulinti, a freelance developer and business development executive at MAN.IT.LLC. MAN.IT.LLC is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of MAN.IT.LLC. 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Abhiram Reddy Mulinti, freelance developer and BDE at MAN.IT.LLC. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for Seamless Infrastructure Development

Dear Hiring Manager,

I came across the job description for a Software Engineer III and was impressed by the role's requirements. As a freelance developer and Business Development Executive at MAN.IT.LLC, I am confident that our company can fulfill your needs and exceed your expectations.

At MAN.IT.LLC, we specialize in providing AI and software consulting services that facilitate the seamless integration of business processes through automated tools. With our expertise in Java programming languages, NoSQL databases (CosmosDB or MongoDB), container orchestration frameworks (Kubernetes), and continuous integration and continuous deployment strategies, we can design, develop, and enhance a reliable and easy-to-maintain infrastructure for your organization.

Our team has extensive experience in telemetry and observability frameworks (Prometheus, Grafana, ELK) and unit and performance testing of applica